In [451]:
import numpy as np
import pandas as pd

In [452]:
# Load data
data = pd.read_csv('D:\crime_data.csv', index_col=0)

In [453]:
data.head()

,Murder,Assault,UrbanPopulation,Rape
State,,,,
Alabama,13.2,236,58,21.2
Alaska,10.0,263,48,44.5
Arizona,8.1,294,80,31.0
Arkansas,8.8,190,50,19.5
California,9.0,276,91,40.6


In [454]:
data.describe()

,Murder,Assault,UrbanPopulation,Rape
count,50.00000,50.000000,50.000000,50.000000
mean,7.78800,170.760000,65.540000,21.232000
std,4.35551,83.337661,14.474763,9.366385
min,0.80000,45.000000,32.000000,7.300000
25%,4.07500,109.000000,54.500000,15.075000
50%,7.25000,159.000000,66.000000,20.100000
75%,11.25000,249.000000,77.750000,26.175000
max,17.40000,337.000000,91.000000,46.000000


In [455]:
# Normalize the data
data = (data - np.mean(data, axis=0)) / np.std(data, axis=0)

In [456]:
#Manhattan Distance
def distance(x1, x2):
    return np.sum(np.abs(x1 - x2))

In [457]:
def k_means_clustering(data, k):
    # Choose initial centroids randomly
    initial_centroids = data.sample(n=k, random_state=1)
    centroids = initial_centroids.copy()

    clusters = {i: [] for i in range(k)}

    # Assign each data point to its nearest centroid
    for index, row in data.iterrows():
        distances = [distance(row[1:], centroids.iloc[i][1:]) for i in range(k)]
        closest_centroid_index = np.argmin(distances)
        clusters[closest_centroid_index].append(index)

    # Update centroids
    for i in range(k):
        centroids.iloc[i] = data.loc[clusters[i]].mean()

    # Continue iterating until maximum number of iterations
    max_iterations = 100
    iteration = 0
    while True:
        
        new_clusters = {i: [] for i in range(k)}

        # Assign each data point to its nearest centroid
        for index, row in data.iterrows():
            distances = [distance(row[1:], centroids.iloc[i][1:]) for i in range(k)]
            closest_centroid_index = np.argmin(distances)
            new_clusters[closest_centroid_index].append(index)

        # converge if no points have changed clusters
        if new_clusters == clusters:
            break

        # Otherwise, update clusters and centroids
        clusters = new_clusters
        for i in range(k):
            centroids.iloc[i] = data.loc[clusters[i]].mean()

        # Check if maximum number of iterations reached
        iteration += 1
        if iteration >= max_iterations:
            break

    return clusters, centroids 

In [458]:
# Get user input for number of clusters
k = int(input('Enter the number of clusters (k): '))

In [459]:
# Cluster the data
clusters, centroids = k_means_clustering(data, k)

In [460]:
# Print clusters
for i in range(k):
    cluster_indices = clusters[i]
    cluster_data = data.loc[cluster_indices]
    cluster_states = cluster_data.index.tolist()
    print(f'Cluster {i+1}:\n {cluster_states}')

    
# Detect and remove outliers using IQR
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1
outliers = ((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))).any(axis=1)
if outliers.any():
    print("\nOutlier records:")
    print(data[outliers])
    data = data[~outliers]  

Cluster 1:
 ['Alabama', 'Alaska', 'Arizona', 'California', 'Colorado', 'Delaware', 'Florida', 'Georgia', 'Illinois', 'Louisiana', 'Maryland', 'Michigan', 'Missouri', 'Nevada', 'New Mexico', 'New York', 'Oregon', 'South Carolina', 'Tennessee', 'Texas', 'Washington']
Cluster 2:
 ['Arkansas', 'Connecticut', 'Hawaii', 'Idaho', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Maine', 'Massachusetts', 'Minnesota', 'Mississippi', 'Montana', 'Nebraska', 'New Hampshire', 'New Jersey', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Pennsylvania', 'Rhode Island', 'South Dakota', 'Utah', 'Vermont', 'Virginia', 'West Virginia', 'Wisconsin', 'Wyoming']

Outlier records:
          Murder   Assault  UrbanPopulation      Rape
State                                                
Alaska  0.513019  1.118060        -1.224067  2.509424
Nevada  1.023254  0.984726         1.078909  2.671197
